In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import numpy as np

cols_rec0111 = ['CASEID', 'V024', 'V025', 'V106', 'V101', 'V102', 'V113',
                'V115', 'V136', 'V161', 'V190']
cols_rec42=['CASEID','V453', 'V457','V457A', 'V459', 'V462', 'V466', 'V481']
cols_rec43 = ['CASEID', 'HIDX','H34', 'H33D','H33M','H33Y', 'H35','H41B', 'H42']
# Anemia, agrego  H35: Cualquier vacuna en los últimos 2 años forma parte de la campaña
cols_rec44 = ['CASEID','HWIDX','HW53', 'HW57A','HW70']
cols_rec91 = ['CASEID', 'S704N','S490AA', 'S490AB', 'S490AC','S490AD','S490AE', 'S490AF']
#rec42: madres
#variable target: V457A: Nivel de Anemia NUEVA DIRECTRIZ OMS 2024/RM 251-2024-MINSA
#otra que agregué por si acaso es V457: Nivel de anemia RM 363-2022-MINSA

#rec44: hijos
#HW57A: Nivel de Anemia NUEVA DIRECTRIZ OMS 2024/RM 251-2024-MINSA

df0111 = pd.read_csv('REC0111_2024.csv', na_values=[" ", "", "NA", "NaN", "nan"], usecols=cols_rec0111)
df42 = pd.read_csv('REC42_2024.csv',  na_values=[" ", "", "NA", "NaN", "nan"],usecols=cols_rec42)
df91 = pd.read_csv('REC91_2024.csv',  na_values=[" ", "", "NA", "NaN", "nan"],usecols=cols_rec91)
df43 = pd.read_csv('REC43_2024.csv',  na_values=[" ", "", "NA", "NaN", "nan"],usecols=cols_rec43)
df44 = pd.read_csv('REC44_2024.csv', na_values=[" ", "", "NA", "NaN", "nan"], usecols=cols_rec44)

In [ ]:
print("Duplicados en df0111_f:", df0111['CASEID'].duplicated().sum())
print("Duplicados en df42_f:", df42['CASEID'].duplicated().sum())
print("Duplicados en df91_f:", df91['CASEID'].duplicated().sum())
print("Duplicados en df43:", df43['CASEID'].duplicated().sum())
print("Duplicados en df44:", df44['CASEID'].duplicated().sum())

Duplicados en df0111_f: 0
Duplicados en df42_f: 0
Duplicados en df91_f: 0
Duplicados en df43: 2143
Duplicados en df44: 2143


In [ ]:
ids_comunes = set(df0111['CASEID']) & set(df42['CASEID']) & set(df43['CASEID']) & \
              set(df44['CASEID']) & set(df91['CASEID'] )

print(f"Número de registros comunes en TODOS los archivos: {len(ids_comunes)}")

# Filtrar todos los DataFrames a solo los registros que estén en todos
df0111_f = df0111[df0111['CASEID'].isin(ids_comunes)]
df42_f   = df42[df42['CASEID'].isin(ids_comunes)]
df43_f   = df43[df43['CASEID'].isin(ids_comunes)]
df44_f   = df44[df44['CASEID'].isin(ids_comunes)]
df91_f   = df91[df91['CASEID'].isin(ids_comunes)]

Número de registros comunes en TODOS los archivos: 14783


In [ ]:
#Hacemos merge a todos los datos que se encuentran en los 5 datasets en uno solo df_merged
df_merged = df0111_f.merge(df42_f, on='CASEID') \
                    .merge(df43_f, on='CASEID') \
                    .merge(df44_f, on='CASEID') \
                    .merge(df91_f, on='CASEID')

print("Tamaño final del dataset completo (sin nulos):", df_merged.shape)


Tamaño final del dataset completo (sin nulos): (20403, 37)


In [ ]:
df_merged

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20398,598402401 2,1,2,1.0,2.0,3.0,11.0,996.0,3.0,2.0,...,118.0,4.0,34.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,...,121.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,100.0,3.0,-86.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0
20401,598402801 2,1,2,1.0,2.0,2.0,11.0,996.0,4.0,2.0,...,106.0,3.0,-241.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
faltantes = df_merged.isnull().sum()
porcentaje_faltantes = (faltantes / len(df_merged)) * 100

resumen = pd.DataFrame({
    "Faltantes": faltantes,
    "Porcentaje": porcentaje_faltantes.round(2)
}).sort_values(by="Porcentaje", ascending=False)

print(resumen)
#no hay:
#H42 Tomando píldoras de hierro, jarabe
#V459 Tener mosquiteros para dormir (informe del hogar)
#V462 Se lavaron las manos antes de preparar la última comida
#H35: Cualquier vacuna en los últimos 2 años forma parte de la campaña
#H33Y: Vitamina A1 año
#H33M: Vitamina A1 mes
#H33D: Vitamina A1 dia
#H34: Vitamina A en los últimos 6 meses
#H41B: Alguna vez recibió la dosis de vitamina A
#B4:Sexo del niño
#B5:el niño está vivo
#BEDAD: meses de nacimiento
#QULT5: el niño es menor de 5 años

        Faltantes  Porcentaje
V462        20403      100.00
H35         20403      100.00
H42         20403      100.00
V459        20403      100.00
H33M        18142       88.92
H33D        18142       88.92
H33Y        18142       88.92
H34         13652       66.91
H41B         3608       17.68
S704N         812        3.98
HW70          665        3.26
HW53          665        3.26
HW57A         665        3.26
V466          100        0.49
S490AE          5        0.02
S490AF          5        0.02
S490AA          5        0.02
S490AC          5        0.02
S490AD          5        0.02
S490AB          5        0.02
CASEID          0        0.00
V102            0        0.00
V101            0        0.00
V025            0        0.00
V024            0        0.00
V481            0        0.00
HIDX            0        0.00
V457A           0        0.00
V457            0        0.00
V113            0        0.00
V115            0        0.00
V106            0        0.00
V136      

In [ ]:
(df_merged['HW57A'].value_counts()/df_merged['HW57A'].value_counts().sum())*100

,count
HW57A,
4.0,61.267606
3.0,21.765123
9.0,9.818624
2.0,7.027054
1.0,0.121593


In [ ]:
(df_merged['HW57A'].value_counts())

,count
HW57A,
4.0,12093
3.0,4296
9.0,1938
2.0,1387
1.0,24


In [ ]:
# Calcular el porcentaje de valores no nulos por columna
columnas_completas = df_merged.columns[df_merged.isnull().mean() < 0.20]

In [ ]:
columnas_completas

Index(['CASEID', 'V024', 'V025', 'V101', 'V102', 'V106', 'V113', 'V115',
       'V136', 'V161', 'V190', 'V453', 'V457', 'V457A', 'V466', 'V481', 'HIDX',
       'H41B', 'HWIDX', 'HW53', 'HW57A', 'HW70', 'S490AA', 'S490AB', 'S490AC',
       'S490AD', 'S490AE', 'S490AF', 'S704N'],
      dtype='object')

In [ ]:
(df_merged['V457A'].value_counts()/df_merged['V457A'].value_counts().sum())*100

,count
V457A,
4,71.072881
3,19.095231
2,7.435181
9,2.127138
1,0.269568


In [ ]:
columnas_completas

Index(['CASEID', 'V024', 'V025', 'V101', 'V102', 'V106', 'V113', 'V115',
       'V136', 'V161', 'V190', 'V453', 'V457', 'V457A', 'V466', 'V481', 'HIDX',
       'H41B', 'HWIDX', 'HW53', 'HW57A', 'HW70', 'S490AA', 'S490AB', 'S490AC',
       'S490AD', 'S490AE', 'S490AF', 'S704N'],
      dtype='object')

In [ ]:
df_completas = df_merged[columnas_completas]

In [ ]:
(df_completas['V457A'].value_counts()/df_completas['V457A'].value_counts().sum())

,count
V457A,
4,0.710729
3,0.190952
2,0.074352
9,0.021271
1,0.002696


In [ ]:
(df_completas['V457A'].value_counts())

,count
V457A,
4,14501
3,3896
2,1517
9,434
1,55


In [ ]:
df_completas

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20398,598402401 2,1,2,1.0,2.0,3.0,11.0,996.0,3.0,2.0,...,118.0,4.0,34.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,...,121.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,100.0,3.0,-86.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0
20401,598402801 2,1,2,1.0,2.0,2.0,11.0,996.0,4.0,2.0,...,106.0,3.0,-241.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
df_completas.count()

,0
CASEID,20403
V024,20403
V025,20403
V101,20403
V102,20403
V106,20403
V113,20403
V115,20403
V136,20403
V161,20403


In [ ]:
#Evitamos repetidos en los hijos
df_limpio = df_completas[df_completas['HIDX'] == df_completas['HWIDX']].copy()

In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20398,598402401 2,1,2,1.0,2.0,3.0,11.0,996.0,3.0,2.0,...,118.0,4.0,34.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,...,121.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,100.0,3.0,-86.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0
20401,598402801 2,1,2,1.0,2.0,2.0,11.0,996.0,4.0,2.0,...,106.0,3.0,-241.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
df_limpio = df_limpio.dropna()

In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,HW53,HW57A,HW70,S490AA,S490AB,S490AC,S490AD,S490AE,S490AF,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,143.0,4.0,-88.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,128.0,4.0,-167.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,125.0,4.0,-78.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,104.0,2.0,-27.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,121.0,4.0,-119.0,1.0,1.0,1.0,0.0,1.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20390,598306701 1,1,2,1.0,2.0,1.0,71.0,10.0,2.0,95.0,...,103.0,3.0,-113.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
20394,598307001 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,111.0,3.0,60.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,...,121.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,100.0,3.0,-86.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0


In [ ]:
df_limpio.size

371432

In [ ]:
df_limpio.drop('HIDX', axis=1, inplace=True)

/tmp/ipython-input-4144911204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio.drop('HIDX', axis=1, inplace=True)


In [ ]:
df_limpio.describe().T

,count,mean,std,min,25%,50%,75%,max
V024,12808.0,13.895456,6.913192,1.0,9.0,15.0,20.0,25.0
V025,12808.0,1.266552,0.442173,1.0,1.0,1.0,2.0,2.0
V101,12808.0,13.895456,6.913192,1.0,9.0,15.0,20.0,25.0
V102,12808.0,1.266552,0.442173,1.0,1.0,1.0,2.0,2.0
V106,12808.0,2.220409,0.700744,0.0,2.0,2.0,3.0,3.0
V113,12808.0,28.807698,27.802785,11.0,11.0,11.0,61.0,97.0
V115,12808.0,800.455497,393.452935,1.0,996.0,996.0,996.0,997.0
V136,12808.0,4.858526,1.784074,2.0,4.0,4.0,6.0,19.0
V161,12808.0,4.569176,11.065210,1.0,2.0,2.0,3.0,97.0
V190,12808.0,2.534588,1.298824,1.0,1.0,2.0,4.0,5.0


In [ ]:
(df_limpio['HW57A'].value_counts()/df_limpio['HW57A'].value_counts().sum())*100

,count
HW57A,
4.0,61.719238
3.0,20.861961
9.0,10.719863
2.0,6.581824
1.0,0.117114


In [ ]:
df_limpio['HW57A'].value_counts()

,count
HW57A,
4.0,7905
3.0,2672
9.0,1373
2.0,843
1.0,15


In [ ]:
df_limpio['V457A'].value_counts()

,count
V457A,
4,9350
3,2331
2,948
9,139
1,40


In [ ]:
# Columnas binarias a limpiar
binarias = ['S490AC','S490AE','S490AB','S490AF','S490AD','S490AA']

In [ ]:
df_limpio[binarias].sum()

,0
S490AC,9301.0
S490AE,9622.0
S490AB,3624.0
S490AF,4267.0
S490AD,3963.0
S490AA,9874.0


In [ ]:
df_limpio['indice_lavado_manos'] = df_limpio[binarias].sum(axis=1)

/tmp/ipython-input-2064859319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['indice_lavado_manos'] = df_limpio[binarias].sum(axis=1)


In [ ]:
df_limpio.drop(df_limpio[binarias], axis=1, inplace=True)

/tmp/ipython-input-380076668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio.drop(df_limpio[binarias], axis=1, inplace=True)


In [ ]:
#Ahora se va a promediar los valores
#HW1
#HW2
#HW3
#HW53: Nivel de hemoglobina (g /dl -1 decimal)
#HW57A:Nivel de Anemia NUEVA DIRECTRIZ OMS 2024/RM 251-2024-MINSA
cols_antropo  = ['HW53','HW70']
col_anemia  = ['HW57A']
cols_constantes = [
    'CASEID', 'V024', 'V025', 'V101', 'V102', 'V106', 'V113', 'V115',
    'V136', 'V161', 'V190', 'V453', 'V457', 'V457A', 'V466', 'V481',
     'indice_lavado_manos', 'S704N'
]

In [ ]:
# Promedios por CASEID
df_promedios = df_limpio.groupby('CASEID')[cols_antropo].mean().reset_index()

In [ ]:
df_promedios

,CASEID,HW53,HW70
0,325503101 2,143.0,-88.0
1,325504701 2,128.0,-167.0
2,325505001 1,125.0,-78.0
3,325508901 2,104.0,-27.0
4,325509701 2,121.0,-119.0
...,...,...,...
11571,598306701 1,103.0,-113.0
11572,598307001 2,111.0,60.0
11573,598402501 1,121.0,1.0
11574,598402701 2,100.0,-86.0


In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,V457A,V466,V481,H41B,HWIDX,HW53,HW57A,HW70,S704N,indice_lavado_manos
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,143.0,4.0,-88.0,1.0,3.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,128.0,4.0,-167.0,2.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,125.0,4.0,-78.0,1.0,2.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,3,1.0,1,0.0,1,104.0,2.0,-27.0,2.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,121.0,4.0,-119.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20390,598306701 1,1,2,1.0,2.0,1.0,71.0,10.0,2.0,95.0,...,4,1.0,1,1.0,1,103.0,3.0,-113.0,1.0,3.0
20394,598307001 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,3,1.0,1,0.0,1,111.0,3.0,60.0,1.0,2.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,121.0,4.0,1.0,3.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,100.0,3.0,-86.0,2.0,2.0


In [ ]:
df_anemia = df_limpio.groupby('CASEID')[col_anemia].max().reset_index()

In [ ]:
df_anemia

,CASEID,HW57A
0,325503101 2,4.0
1,325504701 2,4.0
2,325505001 1,4.0
3,325508901 2,2.0
4,325509701 2,4.0
...,...,...
11571,598306701 1,3.0
11572,598307001 2,3.0
11573,598402501 1,4.0
11574,598402701 2,3.0


In [ ]:
(df_anemia['HW57A'].value_counts()/df_anemia['HW57A'].value_counts().sum())

,count
HW57A,
4.0,0.628887
3.0,0.197650
9.0,0.116189
2.0,0.056410
1.0,0.000864


In [ ]:
df_anemia['HW57A'].value_counts()

,count
HW57A,
4.0,7280
3.0,2288
9.0,1345
2.0,653
1.0,10


In [ ]:
df_limpio.T

,0,1,2,3,4,5,8,11,15,22,...,20381,20382,20386,20387,20388,20390,20394,20399,20400,20401
CASEID,325503101 2,325504701 2,325505001 1,325508901 2,325509701 2,325600801 2,325600801 2,325603201 3,325800601 4,325809301 1,...,598219101 2,598219401 2,598304301 2,598304601 2,598306201 2,598306701 1,598307001 2,598402501 1,598402701 2,598402801 2
V024,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
V025,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
V101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
V102,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
V106,2.0,2.0,1.0,2.0,3.0,3.0,3.0,3.0,1.0,2.0,...,2.0,1.0,3.0,2.0,3.0,1.0,2.0,3.0,2.0,2.0
V113,11.0,71.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,...,71.0,41.0,11.0,11.0,71.0,71.0,11.0,71.0,11.0,11.0
V115,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,...,996.0,15.0,996.0,996.0,996.0,10.0,996.0,996.0,996.0,996.0
V136,4.0,4.0,3.0,4.0,4.0,4.0,4.0,7.0,5.0,7.0,...,6.0,10.0,5.0,7.0,3.0,2.0,3.0,3.0,3.0,4.0
V161,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,95.0,2.0,2.0,2.0,2.0


In [ ]:
print("Duplicados en df43_f:", (df_limpio[cols_constantes]).duplicated().sum())

Duplicados en df43_f: 1232


In [ ]:
df_limpio.to_csv("datos_limpios.csv", index=False)

In [ ]:
df_constantes = df_limpio[cols_constantes].drop_duplicates(subset='CASEID')

In [ ]:
print("Duplicados en df43_f:", (df_constantes['CASEID']).duplicated().sum())

Duplicados en df43_f: 0


In [ ]:
df_constantes

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,V190,V453,V457,V457A,V466,V481,indice_lavado_manos,S704N
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,2,147,4,4,1.0,1,3.0,1.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,2,143,4,4,1.0,1,2.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,2,139,4,4,1.0,1,2.0,1.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,3,129,3,3,1.0,1,2.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,4,144,4,4,1.0,1,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20390,598306701 1,1,2,1.0,2.0,1.0,71.0,10.0,2.0,95.0,1,124,4,4,1.0,1,3.0,1.0
20394,598307001 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,2,119,3,3,1.0,1,2.0,1.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,2,125,4,4,1.0,1,3.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,1,138,4,4,1.0,1,2.0,2.0


In [ ]:
df_final = df_constantes.merge(df_promedios, on='CASEID').merge(df_anemia, on='CASEID')

In [ ]:
df_final.to_csv("anemia_datos_completos.csv", index=False)

PREPROCESAMIENTO

In [ ]:
df_final.T

,0,1,2,3,4,5,6,7,8,9,...,11566,11567,11568,11569,11570,11571,11572,11573,11574,11575
CASEID,325503101 2,325504701 2,325505001 1,325508901 2,325509701 2,325600801 2,325603201 3,325800601 4,325809301 1,325903201 2,...,598219101 2,598219401 2,598304301 2,598304601 2,598306201 2,598306701 1,598307001 2,598402501 1,598402701 2,598402801 2
V024,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
V025,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
V101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
V102,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
V106,2.0,2.0,1.0,2.0,3.0,3.0,3.0,1.0,2.0,3.0,...,2.0,1.0,3.0,2.0,3.0,1.0,2.0,3.0,2.0,2.0
V113,11.0,71.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,71.0,...,71.0,41.0,11.0,11.0,71.0,71.0,11.0,71.0,11.0,11.0
V115,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,996.0,...,996.0,15.0,996.0,996.0,996.0,10.0,996.0,996.0,996.0,996.0
V136,4.0,4.0,3.0,4.0,4.0,4.0,7.0,5.0,7.0,4.0,...,6.0,10.0,5.0,7.0,3.0,2.0,3.0,3.0,3.0,4.0
V161,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,95.0,2.0,2.0,2.0,2.0


In [ ]:
df_final.describe().T #para la transpuesta

,count,mean,std,min,25%,50%,75%,max
V024,11576.0,13.828179,6.909168,1.0,9.0,15.0,19.0,25.0
V025,11576.0,1.262008,0.439746,1.0,1.0,1.0,2.0,2.0
V101,11576.0,13.828179,6.909168,1.0,9.0,15.0,19.0,25.0
V102,11576.0,1.262008,0.439746,1.0,1.0,1.0,2.0,2.0
V106,11576.0,2.232464,0.696765,0.0,2.0,2.0,3.0,3.0
V113,11576.0,28.681928,27.783157,11.0,11.0,11.0,61.0,97.0
V115,11576.0,803.888735,390.892945,1.0,996.0,996.0,996.0,997.0
V136,11576.0,4.779026,1.753214,2.0,4.0,4.0,6.0,19.0
V161,11576.0,4.537578,11.057727,1.0,2.0,2.0,3.0,97.0
V190,11576.0,2.558138,1.298109,1.0,1.0,2.0,4.0,5.0


In [ ]:
(df_limpio['HW57A'].value_counts()/df_limpio['HW57A'].count())*100

,count
HW57A,
4.0,61.719238
3.0,20.861961
9.0,10.719863
2.0,6.581824
1.0,0.117114


In [ ]:
df_limpio.shape

(12808, 23)

In [ ]:
print("Duplicados en df21:", df_final['CASEID'].duplicated().sum())

Duplicados en df21: 0


In [ ]:
df_limpio

,CASEID,V024,V025,V101,V102,V106,V113,V115,V136,V161,...,V457A,V466,V481,H41B,HWIDX,HW53,HW57A,HW70,S704N,indice_lavado_manos
0,325503101 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,143.0,4.0,-88.0,1.0,3.0
1,325504701 2,1,1,1.0,1.0,2.0,71.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,128.0,4.0,-167.0,2.0,2.0
2,325505001 1,1,1,1.0,1.0,1.0,11.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,125.0,4.0,-78.0,1.0,2.0
3,325508901 2,1,1,1.0,1.0,2.0,11.0,996.0,4.0,2.0,...,3,1.0,1,0.0,1,104.0,2.0,-27.0,2.0,2.0
4,325509701 2,1,1,1.0,1.0,3.0,11.0,996.0,4.0,2.0,...,4,1.0,1,0.0,1,121.0,4.0,-119.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20390,598306701 1,1,2,1.0,2.0,1.0,71.0,10.0,2.0,95.0,...,4,1.0,1,1.0,1,103.0,3.0,-113.0,1.0,3.0
20394,598307001 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,3,1.0,1,0.0,1,111.0,3.0,60.0,1.0,2.0
20399,598402501 1,1,2,1.0,2.0,3.0,71.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,121.0,4.0,1.0,3.0,3.0
20400,598402701 2,1,2,1.0,2.0,2.0,11.0,996.0,3.0,2.0,...,4,1.0,1,0.0,1,100.0,3.0,-86.0,2.0,2.0
